In [ ]:
import brightway2 as bw

In [ ]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [ ]:
import sensitivity as st
import Monte_Carlo as mc
import reload_lib as rl

In [ ]:
# Importing self-made libaries
import standards as s
import life_cycle_assessment as lc
import LCA_plots as lp
import box_plot as bp
import reload_lib as rl


lib = [s, mc, lc, lp, bp, st]

In [ ]:
# Path to where the code is stored
path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care'
# Specifying the LCIA method
lcia_method = 'recipe'
rl.reload_lib(lib)

In [ ]:
rl.reload_lib(lib)
flow_legend, database_name, file_name, sheet_name, save_dir, initialization, file_name_unique, db_type = lc.initilization(path, lcia_method)
# Setting up an empty dictionary with the flows as the key

project_name = initialization[0]
flows = initialization[2]
all_acts, eidb, eidb_db = lc.database_initialization(db_type, database_name, project_name)



size = len(flows)



In [ ]:
# Obtaining all the product codes for the process'
func_unit = {}
procces_keys = {}

for act in all_acts:
    for proc in range(size):
        if act['name'] == flows[proc]:
            for sub in act.exchanges():
                if sub['type'] == 'technosphere':
                    func_unit[sub.input] = sub['amount']
                    procces_keys[sub.input] = sub.uncertainty
                    # if sub.uncertainty != {}:
                    # procces_keys[flows[proc]].update({sub.input : sub.uncertainty})
               

           

func_unit

In [ ]:
gwp_cat = lc.lcia_method(initialization[3])
gwp_cat = gwp_cat[1]
gwp_cat

In [ ]:
from copy import deepcopy as dc
import bw2calc as bc
import numpy as np


func_unit_uncert = dc(func_unit)

lcia_results = {key : [] for key in func_unit.keys()}
results = {key : {} for key in func_unit.keys()}

iterations = 1000
for itt in range(iterations):
    print(f"Iteration {itt + 1} of {iterations}")
    for act, val in func_unit.items():
        # if math.isnan(uncert['loc']) == False and math.isnan(uncert['scale']) == False and func_unit_uncert[key] != {}: 
        uncert = procces_keys[act]
        uncert_val = np.random.lognormal(uncert['scale'], np.exp(uncert['loc']))
        func_unit_uncert[act] = uncert_val
        print(f'Perfoming Monte Carlo  for {act}, val = {uncert_val}')
        # # Use the sampled data in the Monte Carlo LCA

        MC_lca = bc.MonteCarloLCA(func_unit_uncert, gwp_cat)
        MC_lca.lci()

        # Initialize cf_params if not already set
        if not hasattr(MC_lca, 'cf_params'):
            MC_lca.cf_params = MC_lca.load_lcia_data()
            # print('redo cf_params')

        # Rebuild the characterization matrix if it's not already initialized
        if not hasattr(MC_lca, 'characterization_matrix'):
            MC_lca.rebuild_characterization_matrix(MC_lca.method)
            # print('rebuild c matric')

        # Perform LCIA calculation directly
        MC_lca.lcia_calculation()

        results[act].update({f"itteration {itt}" : MC_lca.score})
        # print(act, func_unit_uncert[key][act])
    


In [ ]:
import bw2calc as bc



In [ ]:
import bw2data as bd
mc_functional_unit, mc_data_objs, _ = bd.prepare_lca_inputs(
    func_unit,
    method=gwp_cat,
)


In [ ]:

# 1. Define your foreground system (deterministic)
foreground_demand = {}

for act, val in func_unit.items():
    foreground_demand[(project_name, act)] = val

# 2. Specify the method (e.g., Global Warming Potential, IPCC 2013)

# 3. Create the LCA object with distributions enabled for the background
lca = bc.LCA(demand=func_unit, method=gwp_cat)
lca.lci()
lca.lcia()

# 4. Run the Monte Carlo simulation
iterations = 100  # Number of Monte Carlo runs
results = []

# for _ in range(iterations):
#     lca.redo_lcia()  # Resample the background uncertainty
#     results.append(lca.score)  # Store the impact score for each iteration

# # Results now contain 100 Monte Carlo samples
# print(results)


In [ ]:
lca.tech_params